In [1]:
import sqlite3
import pickle
import pandas as pd

In [2]:
shutuba_table = pd.read_pickle("nitiyo_score")

In [4]:
location = {'01' : '札幌',
             '02' : '函館',
             '03' : '福島',
             '04' : '新潟',
             '05' : '東京',
             '06' : '中山',
             '07' : '中京' ,
             '08' : '京都',
             '09' : '阪神',
             '10' : '小倉', }

In [5]:
def remove_leading_zero(input_string):
    if input_string.startswith('0'):
        return str(int(input_string))
    else:
        return input_string

In [6]:
shutuba_table = shutuba_table.astype(str) # 全体を文字列に
shutuba_table.reset_index(inplace=True)
shutuba_table = shutuba_table.rename(columns={'index': 'race_id'}) # インデックスを列に
shutuba_table['race_num'] = shutuba_table['race_id'].str[-2:] # race_num列を追加
shutuba_table['race_num'] = shutuba_table['race_num'].apply(remove_leading_zero) # レース番号を0からはじまらないように変更
shutuba_table['開催'] = shutuba_table['開催'].replace(location) # 開催地をIDから地名

In [7]:
print(shutuba_table)

          race_id 枠番  馬番     単勝    horse_id         馬名  騎手名  性 体重変化 年齢  \
0    202303020801  1   1    6.6  2017105135    アコルドエール   石神  セ   +4  6   
1    202303020801  2   2    6.7  2015105033    オウケンムーン    伴  牡    0  8   
2    202303020801  3   3  125.8  2017102751  ワンダーサーイター   草野  牡   -2  6   
3    202303020801  4   4   17.4  2019102754   ダイゴシンリュウ   中村  牡    0  4   
4    202303020801  4   5  215.6  2020105279  セイカベラトリクス   水沼  牝   -2  3   
..            ... ..  ..    ...         ...        ...  ... ..  ... ..   
484  202301010212  6  10   16.0  2020103455     セラドナイト   武豊  牝   +2  3   
485  202301010212  7  11   17.4  2019105088     マカンマカン   浜中  牡  +12  4   
486  202301010212  7  12   67.1  2019100195    ウインバリオス  角田和  牡  -20  4   
487  202301010212  8  13  183.9  2019100794    マイネルエール  小林勝  セ    0  4   
488  202301010212  8  14   10.7  2020102907   ステラバルセロナ   丹内  牝   +8  3   

           date race_class  race_name  開催 start_time        oddsrank_score  \
0    2023-07-23         障害  3歳以上障

In [8]:
conn = sqlite3.connect('races.sqlite')
cursor = conn.cursor()

In [58]:
# # テスト用：DBのデータ全削除
# cursor.execute('DELETE FROM races')
# cursor.execute('DELETE FROM horses')
# cursor.execute('DELETE FROM runners')
# cursor.execute('DELETE FROM predicts')

******************

racesテーブル

In [9]:
race_table = shutuba_table.drop_duplicates(subset='race_id')[['race_id', 'date', 'race_name', 'race_num', '開催', 'start_time']]

In [10]:
race_table.rename(columns={'race_id' : 'id', 
                           'race_name' : 'name',
                           'race_num' : 'number',
                           '開催' : 'location',
                           'start_time' : 'time'}, 
                  inplace=True)

In [11]:
print(race_table)

               id        date       name number location   time
0    202303020801  2023-07-23  3歳以上障害未勝利      1       福島  10:10
13   202303020802  2023-07-23      3歳未勝利      2       福島  10:45
28   202303020803  2023-07-23      2歳未勝利      3       福島  11:15
42   202303020804  2023-07-23      3歳未勝利      4       福島  11:45
58   202303020805  2023-07-23       2歳新馬      5       福島  12:35
68   202303020806  2023-07-23       2歳新馬      6       福島  13:05
79   202303020807  2023-07-23      3歳未勝利      7       福島  13:35
94   202303020808  2023-07-23  3歳以上1勝クラス      8       福島  14:05
108  202303020809  2023-07-23       横手特別      9       福島  14:36
123  202303020810  2023-07-23       白河特別     10       福島  15:11
138  202303020811  2023-07-23      安達太良S     11       福島  15:45
149  202303020812  2023-07-23  3歳以上1勝クラス     12       福島  16:30
163  202307030801  2023-07-23      2歳未勝利      1       中京  10:01
171  202307030802  2023-07-23      2歳未勝利      2       中京  10:35
184  202307030803  2023-07-23      3歳未勝利

In [12]:
race_table.to_sql('races', conn, if_exists='append', index=None)

36

*************

horsesテーブル

In [13]:
horse_table = shutuba_table.drop_duplicates(subset='horse_id')[['horse_id', '馬名', '年齢', '性']]

In [14]:
horse_table.rename(columns={'horse_id' : 'id', 
                           '馬名' : 'name',
                           '年齢' : 'age',
                           '性' : 'gender'}, 
                  inplace=True)

In [15]:
old = pd.read_sql_query("SELECT id FROM horses", conn)  # 登録済みのidを取得
mask = ~horse_table.iloc[:, 0].isin(old.iloc[:, 0])
new_horse_table = horse_table[mask]

In [16]:
new_horse_table.to_sql('horses', conn, if_exists='append', index=None)

489

**************

runnersテーブル

In [17]:
runner_table = shutuba_table[['race_id', '馬番', '枠番', '騎手名', '単勝', 'horse_id']].copy()

In [18]:
runner_table.rename(columns={'馬番' : 'number', 
                             '枠番' : 'position',
                             '騎手名' : 'jockey',
                             '単勝' : 'odds'}, 
                  inplace=True)

In [19]:
runner_table.to_sql('runners', conn, if_exists='append', index=None)

489

**************

In [70]:
# cursor.execute('SELECT * FROM runners')
# data = cursor.fetchall()

# # データを表示
# for row in data:
#     print(row)

('202303020701', '1', '1', '柴田大', '19.5', '2021106804')
('202303020701', '2', '2', '石川', '20.9', '2021102302')
('202303020701', '3', '2', '大野', '13.1', '2021105297')
('202303020701', '4', '3', '菊沢', '135.4', '2021103343')
('202303020701', '5', '3', '永野', '2.6', '2021106866')
('202303020701', '6', '4', '菅原明', '7.5', '2021105060')
('202303020701', '7', '4', '原田和', '216.2', '2021107294')
('202303020701', '8', '5', '津村', '24.7', '2021102391')
('202303020701', '9', '5', '藤田菜', '11.7', '2021101023')
('202303020701', '10', '6', '原', '141.6', '2021101152')
('202303020701', '11', '6', '戸崎圭', '4.6', '2021105057')
('202303020701', '12', '7', 'Ｍデムーロ', '14.0', '2021104127')
('202303020701', '13', '7', '小林美', '16.2', '2021100236')
('202303020701', '14', '8', '伊藤', '93.3', '2021104618')
('202303020701', '15', '8', '木幡巧', '94.7', '2021100746')
('202303020702', '1', '1', '津村', '2.9', '2021106963')
('202303020702', '2', '2', '木幡巧', '8.0', '2021104112')
('202303020702', '3', '3', '菊沢', '14.6', '202110312

In [20]:
conn.close()